In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
a_pos_5289 = "/content/drive/My Drive/npy File All/A.thaliana5289_pos.fasta-ENAC-Len-5.npy"
a_neg_5289 = "/content/drive/My Drive/npy File All/A.thaliana5289_neg.fasta-ENAC-Len-5.npy"
a_pos_1000 = "/content/drive/My Drive/npy File All/A.thaliana1000indep_pos-ENAC-Len-5.npy"
a_neg_1000 = "/content/drive/My Drive/npy File All/A.thaliana1000indep_neg.fasta-ENAC-5.npy"

a_x_train_pos = np.load(a_pos_5289) 
a_x_train_neg = np.load(a_neg_5289) 
a_x_test_pos = np.load(a_pos_1000) 
a_x_test_neg = np.load(a_neg_1000) 


b_pos_5289 =  "/content/drive/My Drive/npy File All/A.thaliana5289_pos.fasta-Kmer-1.npy"
b_neg_5289 = "/content/drive/My Drive/npy File All/A.thaliana5289_neg.fasta-Kmer-1.npy"
b_pos_1000 = "/content/drive/My Drive/npy File All/A.thaliana1000indep_pos-Kmer-1.npy"
b_neg_1000 = "/content/drive/My Drive/npy File All/A.thaliana1000indep_neg.fasta-kmer-1.npy"


b_x_train_pos = np.load(b_pos_5289) 
b_x_train_neg = np.load(b_neg_5289) 
b_x_test_pos = np.load(b_pos_1000) 
b_x_test_neg = np.load(b_neg_1000) 

y_train_pos = np.tile(1, 5289) 
y_train_neg = np.tile(0, 5289) 
y_test_pos = np.tile(1, 1000) 
y_test_neg = np.tile(0, 1000)

print(a_x_test_neg.shape)
print(a_x_train_neg.shape)
print(a_x_test_pos.shape)
print(a_x_train_pos.shape)
print(b_x_test_neg.shape)
print(b_x_train_neg.shape)
print(b_x_test_pos.shape)
print(b_x_train_pos.shape)

(1000, 148)
(5289, 148)
(1000, 148)
(5289, 148)
(1000, 4)
(5289, 4)
(1000, 4)
(5289, 4)


In [4]:
a_x_training = np.concatenate((a_x_train_pos, a_x_train_neg), axis = 0)
a_x_test = np.concatenate((a_x_test_pos, a_x_test_neg), axis = 0)

b_x_training = np.concatenate((b_x_train_pos, b_x_train_neg), axis = 0)
b_x_test = np.concatenate((b_x_test_pos, b_x_test_neg), axis = 0)

y_training = np.concatenate((y_train_pos,y_train_neg))
y_test = np.concatenate((y_test_pos,y_test_neg))

In [5]:
x_training = np.concatenate((a_x_training, b_x_training), axis = 1)

x_test = np.concatenate((a_x_test, b_x_test), axis = 1)

In [6]:
seed = 40
np.random.seed(seed)

kf = KFold(n_splits=5, random_state=seed, shuffle=True)

In [7]:

test_avg_acc_list = []
j = 1
for c in range(1,452,50):
  RFC = RandomForestClassifier(n_estimators=c,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 1
Train Accuracy :  0.9105412432049161 	Validation Accuracy :  0.7542533081285444 	Test Accuracy  0.5845
Train Accuracy :  0.904632474592295 	Validation Accuracy :  0.7386578449905482 	Test Accuracy  0.593
Train Accuracy :  0.9145592058614984 	Validation Accuracy :  0.7651228733459358 	Test Accuracy  0.585
Train Accuracy :  0.9190594351884674 	Validation Accuracy :  0.7881796690307329 	Test Accuracy  0.579
Train Accuracy :  0.9201228878648233 	Validation Accuracy :  0.774468085106383 	Test Accuracy  0.6625
Avg Test Accuracy: 0.6008
Epoch 2/10 for C = 51
Train Accuracy :  1.0 	Validation Accuracy :  0.9494328922495274 	Test Accuracy  0.7255
Train Accuracy :  1.0 	Validation Accuracy :  0.9465973534971645 	Test Accuracy  0.7335
Train Accuracy :  1.0 	Validation Accuracy :  0.9607750472589792 	Test Accuracy  0.7195
Train Accuracy :  1.0 	Validation Accuracy :  0.9498817966903074 	Test Accuracy  0.7295
Train Accuracy :  1.0 	Validation Accuracy :  0.9536643026004729 	Tes

In [9]:
test_avg_acc_list = []
j = 1
for c in range(395,405,1):
  RFC = RandomForestClassifier(n_estimators=c,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 395
Train Accuracy :  1.0 	Validation Accuracy :  0.9636105860113422 	Test Accuracy  0.7505
Train Accuracy :  1.0 	Validation Accuracy :  0.9551039697542533 	Test Accuracy  0.7295
Train Accuracy :  1.0 	Validation Accuracy :  0.9697542533081286 	Test Accuracy  0.7415
Train Accuracy :  1.0 	Validation Accuracy :  0.9617021276595744 	Test Accuracy  0.731
Train Accuracy :  1.0 	Validation Accuracy :  0.9626477541371158 	Test Accuracy  0.744
Avg Test Accuracy: 0.7392999999999998
Epoch 2/10 for C = 396
Train Accuracy :  1.0 	Validation Accuracy :  0.9617202268431002 	Test Accuracy  0.734
Train Accuracy :  1.0 	Validation Accuracy :  0.9527410207939508 	Test Accuracy  0.7235
Train Accuracy :  1.0 	Validation Accuracy :  0.9716446124763705 	Test Accuracy  0.732
Train Accuracy :  1.0 	Validation Accuracy :  0.9583924349881797 	Test Accuracy  0.734
Train Accuracy :  1.0 	Validation Accuracy :  0.9640661938534278 	Test Accuracy  0.734
Avg Test Accuracy: 0.7314999999999999
Epoc

In [10]:
test_avg_acc_list = []
j = 1
for c in range(1,101,10):
  RFC = RandomForestClassifier(n_estimators=270,max_depth=c)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 1
Train Accuracy :  0.7843299456393288 	Validation Accuracy :  0.7915879017013232 	Test Accuracy  0.6275
Train Accuracy :  0.7872843299456393 	Validation Accuracy :  0.7953686200378072 	Test Accuracy  0.642
Train Accuracy :  0.777948475537698 	Validation Accuracy :  0.7698487712665406 	Test Accuracy  0.5905
Train Accuracy :  0.7767930993737445 	Validation Accuracy :  0.7777777777777778 	Test Accuracy  0.645
Train Accuracy :  0.7766749379652605 	Validation Accuracy :  0.7687943262411348 	Test Accuracy  0.628
Avg Test Accuracy: 0.6266
Epoch 2/10 for C = 11
Train Accuracy :  0.9991727723942331 	Validation Accuracy :  0.9574669187145558 	Test Accuracy  0.7355
Train Accuracy :  0.9985818955329709 	Validation Accuracy :  0.9489603024574669 	Test Accuracy  0.723
Train Accuracy :  0.9984637201607185 	Validation Accuracy :  0.9551039697542533 	Test Accuracy  0.7345
Train Accuracy :  0.9987002245066762 	Validation Accuracy :  0.9550827423167849 	Test Accuracy  0.727
Train Accu